In [10]:
import pandas as pd
import numpy as np

# Load the data from the CSV file
data= pd.read_csv("C:/Users/gbo10/OneDrive/research/thesis and paper/telemetry files/GX010067_HERO11 Black-ACCL (1).csv")


# Convert the 'date' column to datetime format
data['date'] = pd.to_datetime(data['date'])

# Calculate time intervals in seconds using the 'date' column
data['time_intervals'] = data['date'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds()

# Display the initial timestamp in the data
initial_timestamp = data['date'].min()

# Adjust the start and end times based on the initial timestamp
start_time = initial_timestamp + pd.Timedelta(seconds=2)
end_time = initial_timestamp + pd.Timedelta(seconds=54)

# Filter the data within the specified time range
filtered_time_data = data[(data['date'] >= start_time) & (data['date'] <= end_time)].copy()

# Calculate cumulative speed using every 30 frames
frame_rate = 30  # assuming 30 frames per second
filtered_time_data['frame_index'] = filtered_time_data.index // frame_rate
filtered_time_data = filtered_time_data.groupby('frame_index').agg({
    'date': 'first',
    'Accelerometer [m/s2]': 'mean',
    'time_intervals': 'sum'
}).reset_index()

# Recalculate the speed within this time range
filtered_time_data['total_speed'] = np.cumsum(filtered_time_data['Accelerometer [m/s2]'] * filtered_time_data['time_intervals'])
filtered_time_data['total_speed_kmh'] = filtered_time_data['total_speed'] * 3.6

# Calculate the average speed in km/h within this time range
avg_speed_kmh_filtered_time = filtered_time_data['total_speed_kmh'].mean()

avg_speed_kmh_filtered_time



71.60664942115727

In [194]:
accelration=  0.274





hour_acceleration = accelration * 3.6




# Calculate the average speed in km/h over the entire time range
avg_speed_kmh = hour_acceleration *1 
print(avg_speed_kmh)



0.9864


In [6]:
import pandas as pd

def process_frame_data( luminance_file_path, frame_index, frame_rate=30):
    # Load the CSV files
    luminance_data = pd.read_csv(luminance_file_path)

    # Calculate the video second from the frame index
    video_second = frame_index // frame_rate

    # Extract data for the given second from the accelerometer CSV
    start_frame = video_second * frame_rate
    end_frame = start_frame + frame_rate

    # Calculate the average acceleration in m/s^2 for that second

    # Extract the luminance value for the given frame index
    luminance_value = luminance_data.iloc[frame_index]['Average luminance']

    # Print the results
    print(f"Video second: {video_second}")
    print(f"Luminance value for frame index {frame_index}: {luminance_value}")

# Example usage
luminance_file_path = "C:/Users/gbo10/OneDrive/research/thesis and paper/telemetry files/GX010157_HERO11 Black-YAVG.csv"
frame_index =3942  # Example frame index

process_frame_data( luminance_file_path, frame_index)


Video second: 131
Luminance value for frame index 3942: 70.06074259069328


# extract GPS speed

In [14]:
import gpxpy
import pandas as pd
from geopy.distance import geodesic
import datetime

# Load GPX file
def load_gpx(file_path):
    with open(file_path, 'r') as gpx_file:
        gpx = gpxpy.parse(gpx_file)
    return gpx

# Extract data from GPX file
def extract_data(gpx):
    data = []
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                data.append({
                    'time': point.time,
                    'latitude': point.latitude,
                    'longitude': point.longitude,
                    'elevation': point.elevation,
                })
    return data

# Calculate speed between points
def calculate_speed(data):
    for i in range(1, len(data)):
        p1 = (data[i-1]['latitude'], data[i-1]['longitude'])
        p2 = (data[i]['latitude'], data[i]['longitude'])
        distance = geodesic(p1, p2).meters
        time_diff = (data[i]['time'] - data[i-1]['time']).total_seconds()
        speed_m_s = distance / time_diff if time_diff != 0 else 0
        data[i]['speed_m_s'] = speed_m_s
        data[i]['speed_kmh'] = speed_m_s * 3.6 
    return data

# Process GPX file and extract speeds
def process_gpx(file_path):
    gpx = load_gpx(file_path)
    data = extract_data(gpx)
    data = calculate_speed(data)
    df = pd.DataFrame(data)
    return df

# Main function to run the process
def main(file_path):
    df = process_gpx(file_path)
    # Fill speed for the first point as NaN
    df.at[0, 'speed_m_s'] = None
    df.at[0, 'speed_kmh'] = None
    
    # Calculate elapsed time in seconds from the start
    start_time = df['time'].iloc[0]
    df['elapsed_seconds'] = (df['time'] - start_time).dt.total_seconds().astype(int)
    
    # Group by the elapsed seconds
    df_grouped = df.groupby('elapsed_seconds').mean()
    
    return df_grouped

# Example usage
file_path = "C:/Users/gbo10/Dropbox/research videos/car-current-/may 2024/07.05.2024/GX010178_1_GPS9.gpx"
df = main(file_path)
#group by second
print(df)


GPXException: Invalid value "0", possible: ('none', '2d', '3d', 'dgps', 'pps', '3')

        speed_m_s  speed_kmh
second                      
0        0.867064   3.121431
1        0.880537   3.169934
2        1.098050   3.952980
3        0.805619   2.900229
4        1.029268   3.705364
5        1.011854   3.642675
6        0.819931   2.951753
7        0.917629   3.303466
8        1.685521   6.067875
9        0.826585   2.975705
10       0.716636   2.579889
11       0.810535   2.917925
12       0.964231   3.471231
13       0.765383   2.755379
14       1.055110   3.798396
15       1.345151   4.842543
16       1.231880   4.434768
17       1.033776   3.721592
18       1.608144   5.789319
19       1.486671   5.352015
20       1.567889   5.644400
21       1.344619   4.840627
22       1.129824   4.067365
23       0.805885   2.901185
24       0.933869   3.361930
25       0.929062   3.344624
26       0.868696   3.127305
27       1.165164   4.194591
28       1.214668   4.372804
29       0.894697   3.220910
30       0.891784   3.210423
31       0.837918   3.016504
32       0.665